<a href="https://colab.research.google.com/github/Walls182/LoginPersistenciabd/blob/main/TitanicKaggleClasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Paso 1: Configuración inicial

In [2]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Configuración de visualización
plt.style.use('ggplot')
sns.set_palette("Set2")

##Paso 2: Cargar los datos

In [ ]:
# Subir archivos a Colab
uploaded = files.upload()

# Cargar los datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("Forma del dataset de entrenamiento:", train_df.shape)
print("Forma del dataset de prueba:", test_df.shape)
print("\nPrimeras 5 filas del train:")
print(train_df.head())

##Paso 3: Analisis Exploratorio de los Datos

In [ ]:
# Información básica del dataset
print("Información del dataset de entrenamiento:")
print(train_df.info())
print("\nEstadísticas descriptivas:")
print(train_df.describe())

# Valores faltantes
print("\nValores faltantes en train:")
print(train_df.isnull().sum())
print("\nValores faltantes en test:")
print(test_df.isnull().sum())

# Distribución de la variable objetivo (Survived)
plt.figure(figsize=(8, 5))
sns.countplot(x='Survived', data=train_df)
plt.title('Distribución de Sobrevivientes (0 = No, 1 = Sí)')
plt.show()

# Sobrevivencia por género
plt.figure(figsize=(8, 5))
sns.countplot(x='Survived', hue='Sex', data=train_df)
plt.title('Sobrevivencia por Género')
plt.show()

# Sobrevivencia por clase
plt.figure(figsize=(8, 5))
sns.countplot(x='Survived', hue='Pclass', data=train_df)
plt.title('Sobrevivencia por Clase')
plt.show()

##Paso 4: Preexploracion y procesamiento de los datos

In [ ]:
def preprocess_data(df):
    # Crear una copia para no modificar el original
    df_processed = df.copy()

    # 1. Ingeniería de características - Extraer título del nombre
    df_processed['Title'] = df_processed['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

    # Agrupar títulos poco comunes
    title_mapping = {'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
                    'Dr': 'Rare', 'Rev': 'Rare', 'Col': 'Rare', 'Major': 'Rare',
                    'Mlle': 'Rare', 'Countess': 'Rare', 'Ms': 'Miss', 'Lady': 'Rare',
                    'Jonkheer': 'Rare', 'Don': 'Rare', 'Dona': 'Rare', 'Mme': 'Rare',
                    'Capt': 'Rare', 'Sir': 'Rare'}

    df_processed['Title'] = df_processed['Title'].map(title_mapping)

    # 2. Crear familia size
    df_processed['FamilySize'] = df_processed['SibSp'] + df_processed['Parch'] + 1

    # 3. Crear IsAlone
    df_processed['IsAlone'] = 0
    df_processed.loc[df_processed['FamilySize'] == 1, 'IsAlone'] = 1

    # 4. Manejar valores faltantes en Age
    age_imputer = SimpleImputer(strategy='median')
    df_processed['Age'] = age_imputer.fit_transform(df_processed[['Age']])

    # 5. Manejar valores faltantes en Fare (solo para test)
    if 'Fare' in df_processed.columns:
        fare_imputer = SimpleImputer(strategy='median')
        df_processed['Fare'] = fare_imputer.fit_transform(df_processed[['Fare']])

    # 6. Manejar valores faltantes en Embarked
    df_processed['Embarked'].fillna(df_processed['Embarked'].mode()[0], inplace=True)

    # 7. Codificar variables categóricas
    label_encoder = LabelEncoder()
    df_processed['Sex'] = label_encoder.fit_transform(df_processed['Sex'])
    df_processed['Embarked'] = label_encoder.fit_transform(df_processed['Embarked'])
    df_processed['Title'] = label_encoder.fit_transform(df_processed['Title'])

    # 8. Eliminar columnas no necesarias
    columns_to_drop = ['Name', 'Ticket', 'Cabin', 'PassengerId']
    df_processed.drop([col for col in columns_to_drop if col in df_processed.columns],
                     axis=1, inplace=True)

    return df_processed

# Aplicar preprocesamiento
train_processed = preprocess_data(train_df)
test_processed = preprocess_data(test_df)

print("Train procesado:")
print(train_processed.head())

##Paso 5: Preparacion de los datos para el modelo

In [ ]:
# Separar características y variable objetivo
X = train_processed.drop('Survived', axis=1)
y = train_processed['Survived']

# Dividir en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

##Paso 6: Entrenamiento del modelo para clasificacion

In [ ]:
# Usar Regresión Logística (apropiado para clasificación)
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred = model.predict(X_val_scaled)
y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]  # Probabilidades para la clase 1

# Evaluar el modelo
accuracy = accuracy_score(y_val, y_pred)
print(f'Precisión del modelo: {accuracy:.4f}')
print('\nMatriz de confusión:')
print(confusion_matrix(y_val, y_pred))
print('\nReporte de clasificación:')
print(classification_report(y_val, y_pred))

##Paso 7: Visualizacion de los resultados

In [ ]:
# Matriz de confusión visual
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.ylabel('Verdadero')
plt.xlabel('Predicho')

# Importancia de características
plt.subplot(1, 2, 2)
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.coef_[0]
})
feature_importance = feature_importance.sort_values('importance', ascending=True)
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.title('Importancia de Características')
plt.tight_layout()
plt.show()

##Paso 8: Prediccion del dataset de kaggle de titanic

In [ ]:
# Preparar datos de test
X_test = test_processed
X_test_scaled = scaler.transform(X_test)

# Hacer predicciones
test_predictions = model.predict(X_test_scaled)

# Crear archivo de submission
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': test_predictions
})

# Guardar submission
submission.to_csv('titanic_submission.csv', index=False)
print("Archivo de submission creado: titanic_submission.csv")

# Descargar el archivo
files.download('titanic_submission.csv')

##Paso 9: Hacer las predicciones individuales

In [ ]:
def predecir_sobrevivencia():
    print("Ingrese los datos del pasajero:")
    pclass = int(input("Clase (1, 2, 3): "))
    sex = input("Sexo (male/female): ")
    age = float(input("Edad: "))
    sibsp = int(input("Número de hermanos/esposos a bordo: "))
    parch = int(input("Número de padres/hijos a bordo: "))
    fare = float(input("Tarifa pagada: "))
    embarked = input("Puerto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton): ")

    # Crear DataFrame con los datos
    passenger_data = pd.DataFrame({
        'Pclass': [pclass],
        'Sex': [1 if sex.lower() == 'female' else 0],  # Female=1, Male=0
        'Age': [age],
        'SibSp': [sibsp],
        'Parch': [parch],
        'Fare': [fare],
        'Embarked': [0 if embarked.upper() == 'C' else 1 if embarked.upper() == 'Q' else 2],
        'Title': [2],  # Asumimos Mr como default
        'FamilySize': [sibsp + parch + 1],
        'IsAlone': [1 if (sibsp + parch) == 0 else 0]
    })

    # Reordenar columnas para que coincidan con el entrenamiento
    passenger_data = passenger_data[X.columns]

    # Escalar
    passenger_scaled = scaler.transform(passenger_data)

    # Predecir
    prediction = model.predict(passenger_scaled)
    probability = model.predict_proba(passenger_scaled)[0][1]

    resultado = "SOBREVIVIRÍA" if prediction[0] == 1 else "NO SOBREVIVIRÍA"
    print(f"\nPredicción: El pasajero {resultado}")
    print(f"Probabilidad de sobrevivir: {probability:.2%}")

# Ejecutar la función de predicción
predecir_sobrevivencia()